In [ ]:
import random
import numpy as np
import openai
import pandas as pd
import os
import sys
import time
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import openai
# Add the path to the constants file to the system path
sys.path.append('../../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *
from CF_utils import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, '../data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'ml-1m/merged_data.dat')
print(f'Data path: {data_path}')

# output

CF_output_path = os.path.join(DATA_DIR, 'ml-1m/output/CF_output_path_2_ratings_per_user.dat')
print(f'Data path: {data_path}')

CF_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_CF_output_path_2_ratings_per_user.dat')
print(f'Data path: {data_path}')

In [ ]:
# Read and Merge Data
def load_and_merge_data(movies_path, ratings_path, users_path):
    # Load each file
    movies = pd.read_csv(movies_path, delimiter='::', engine= 'python', header=None, names=['MovieID', 'Title', 'Genres'], encoding='ISO-8859-1')
    ratings = pd.read_csv(ratings_path, delimiter='::', engine= 'python', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'], encoding='ISO-8859-1')
    users = pd.read_csv(users_path,delimiter='::', engine= 'python', header=None, names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], encoding='ISO-8859-1')
    # Merge datasets
    merged_data = pd.merge(pd.merge(ratings, users, on='UserID'), movies, on='MovieID')
    return merged_data


# Filter Users with ≥ 5 Ratings
def filter_users(data):
    user_rating_counts = data['UserID'].value_counts()
    valid_users = user_rating_counts[user_rating_counts >= 5].index.tolist()
    return data[data['UserID'].isin(valid_users)]


# Calculate Pearson Correlation Coefficient
# source RMIT courses
def pearson_correlation(interaction_matrix):
    """
    Compute the Pearson Correlation Coefficient matrix for the user-item interaction matrix.

    Args:
    interaction_matrix (csr_matrix): A sparse matrix where rows represent users and columns represent items.
                                     The values in the matrix are the ratings given by users to items.

    Returns:
    numpy.ndarray: A 2D array representing the Pearson Correlation Coefficients between each pair of users.
    """
    # Convert sparse matrix to dense format for processing
    dense_matrix = interaction_matrix.toarray()
    
    # Get the number of users
    n_users = dense_matrix.shape[0]

    # Initialize the Pearson Correlation matrix
    pearson_corr_matrix = np.zeros((n_users, n_users))

    # Small constant to avoid division by zero
    EPSILON = 1e-9

    # Iterate over each pair of users
    for i in range(n_users):
        for j in range(n_users):
            # Get the rating vectors for the current pair of users
            user_i_vec = dense_matrix[i, :]
            user_j_vec = dense_matrix[j, :]

            # Masks for rated items
            mask_i = user_i_vec > 0
            mask_j = user_j_vec > 0

            # Find indices of corrated items
            corrated_index = np.intersect1d(np.where(mask_i)[0], np.where(mask_j)[0])

            # Skip if no items are corrated
            if len(corrated_index) == 0:
                continue

            # Compute the mean rating for each user over corrated items
            mean_user_i = np.mean(user_i_vec[corrated_index])
            mean_user_j = np.mean(user_j_vec[corrated_index])

            # Compute the deviations from the mean
            user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
            user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

            # Calculate the components for Pearson correlation
            r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
            r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

            r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
            r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

            # Calculate Pearson correlation
            sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

            # Store the similarity in the matrix
            pearson_corr_matrix[i, j] = sim

    return pearson_corr_matrix


# Find Valid Neighbors
def get_valid_neighbors(pcc_matrix, threshold=0.6):
    valid_neighbors = {}
    for i, row in enumerate(pcc_matrix):
        valid_neighbors[i] = np.where(row > threshold)[0]
    return valid_neighbors

In [ ]:
data = pd.read_csv(data_path)
data.head(3)

In [5]:
# Create User-Item Interaction Matrix
interaction_matrix = pd.pivot_table(data, index='UserID', columns='MovieID', values='Rating').fillna(0)
csr_interaction_matrix = csr_matrix(interaction_matrix.values)

# Calculate Pearson Correlation Coefficient Matrix
pcc_matrix = pearson_correlation(csr_interaction_matrix)

pcc_matrix

array([[ 1.        ,  0.41666667, -0.33218192, ...,  0.        ,
         0.05685735, -0.04351941],
       [ 0.41666667,  1.        ,  0.23683386, ..., -0.5       ,
         0.57207755, -0.0271435 ],
       [-0.33218192,  0.23683386,  1.        , ...,  0.5       ,
         0.30927686, -0.39528471],
       ...,
       [ 0.        , -0.5       ,  0.5       , ...,  1.        ,
         0.27116307, -0.39712226],
       [ 0.05685735,  0.57207755,  0.30927686, ...,  0.27116307,
         1.        ,  0.24230884],
       [-0.04351941, -0.0271435 , -0.39528471, ..., -0.39712226,
         0.24230884,  1.        ]])

In [6]:
csr_interaction_matrix

<6040x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [7]:
interaction_matrix

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
import random
import numpy as np
import pandas as pd

def predict_ratings_with_collaborative_filtering_and_save(data, 
                                                          pcc_matrix, 
                                                          user_column_name='reviewerID', 
                                                          movie_column_name='title', 
                                                          movie_id_column='asin',
                                                          rating_column_name='rating', 
                                                          num_ratings_per_user=1, 
                                                          num_similar_users=4, 
                                                          save_path='cf_predictions.csv', 
                                                          seed=RANDOM_STATE):
    """
    Predicts movie ratings using a user-based collaborative filtering approach 
    and saves the predictions to a CSV file.

    Parameters:
    - data: DataFrame containing user ratings.
    - pcc_matrix: Pearson Correlation Coefficient matrix.
    - user_column_name: Column name for user IDs.
    - movie_column_name: Column name for movie titles.
    - movie_id_column: Column name for movie IDs.
    - rating_column_name: Column name for ratings.
    - num_ratings_per_user: Number of historical ratings to consider per similar user.
    - num_similar_users: Number of similar users to consider for prediction.
    - save_path: Path to save the predictions CSV file.
    - seed: Seed for random number generation.

    Returns:
    - DataFrame: DataFrame containing prediction results.
    """

    results = []
    unique_users = data[user_column_name].unique()
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(unique_users)}

    random.seed(seed)

    for user_id in unique_users:
        user_idx = user_id_to_index[user_id]

        # Find the top similar users based on Pearson Correlation Coefficient
        similar_users_idx = np.argsort(-pcc_matrix[user_idx])[:num_similar_users + 1]
        similar_users_idx = similar_users_idx[similar_users_idx != user_idx][:num_similar_users]

        # Display user and similar users
        print(f"\nProcessing User {user_id} (User Index: {user_idx})")
        print(f"  Similar Users: {[unique_users[idx] for idx in similar_users_idx]}")

        # Collect historical ratings of the user
        user_historical_ratings = data[data[user_column_name] == user_id].head(num_ratings_per_user)
        print(f"  User {user_id} - Historical Ratings:")
        for _, hist_row in user_historical_ratings.iterrows():
            print(f"    * Title: {hist_row[movie_column_name]}, Rating: {hist_row[rating_column_name]} stars")

        # Collect historical ratings from similar users
        similar_users_ratings = ""
        for idx in similar_users_idx:
            similar_user_id = unique_users[idx]
            similar_user_data = data[data[user_column_name] == similar_user_id]
            historical_ratings = similar_user_data.head(num_ratings_per_user)
            for _, row in historical_ratings.iterrows():
                rating_info = f"* Title: {row[movie_column_name]}, Rating: {row[rating_column_name]} stars"
                print(f"    Similar User {similar_user_id} - Historical Rating: {rating_info}")
                similar_users_ratings += rating_info + "\n"

        # Select a random movie from the user's data for prediction
        user_data = data[data[user_column_name] == user_id]
        random_movie_row = user_data.sample(n=1, random_state=seed).iloc[0]
        random_movie_title = random_movie_row[movie_column_name]
        random_movie_id = random_movie_row[movie_id_column]
        actual_rating = random_movie_row[rating_column_name]

        # Predict rating using collaborative filtering
        combined_text = f"Title: {random_movie_title}"
        predicted_rating = predict_rating_combined_ChatCompletion(
            combined_text, 
            approach="CF", 
            similar_users_ratings=similar_users_ratings
        )

        # Store prediction results
        results.append([user_id, random_movie_id, random_movie_title, actual_rating, predicted_rating])

    # Save predictions to CSV file and return the results
    results_df = pd.DataFrame(results, columns=['user_id', 'item_id', 'title', 'actual_rating', 'predicted_rating'])
    results_df.to_csv(save_path, index=False)
    print(f"\nPredictions saved to {save_path}")

    return results_df


In [13]:
%%time


cf_predictions = predict_ratings_with_collaborative_filtering_and_save(data, 
                                                                       pcc_matrix, 
                                                                       save_path=CF_output_path,
                                                                       user_column_name='UserID',
                                                                       movie_column_name='Title',
                                                                       movie_id_column='MovieID',
                                                                       rating_column_name='Rating', 
                                                                       num_ratings_per_user=2,
                                                                       num_similar_users=4, )



Processing User 1 (User Index: 0)
  Similar Users: [4871, 74, 503, 2099]
  User 1 - Historical Ratings:
    * Title: One Flew Over the Cuckoo's Nest (1975), Rating: 5 stars
    * Title: James and the Giant Peach (1996), Rating: 3 stars
    Similar User 4871 - Historical Rating: * Title: Back to the Future (1985), Rating: 4 stars
    Similar User 4871 - Historical Rating: * Title: Sixth Sense, The (1999), Rating: 5 stars
    Similar User 74 - Historical Rating: * Title: Erin Brockovich (2000), Rating: 4 stars
    Similar User 74 - Historical Rating: * Title: Titanic (1997), Rating: 3 stars
    Similar User 503 - Historical Rating: * Title: Mission: Impossible (1996), Rating: 2 stars
    Similar User 503 - Historical Rating: * Title: Gladiator (2000), Rating: 5 stars
    Similar User 2099 - Historical Rating: * Title: James and the Giant Peach (1996), Rating: 3 stars
    Similar User 2099 - Historical Rating: * Title: Airplane! (1980), Rating: 4 stars
Constructed Prompt for CF approach:

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Read the data
data = pd.read_csv(CF_output_path)

# Display the original data types
print("Original Data Types:")
print(data.dtypes)
print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
# Load the original CF predictions
cf_data = pd.read_csv(CF_output_path)
cf_data

In [8]:
def identify_and_rerun_failed_cf_predictions(data, 
                                             pcc_matrix, 
                                             user_column_name='user_id', 
                                             movie_column_name='title', 
                                             movie_id_column='item_id',
                                             rating_column_name='actual_rating', 
                                             num_ratings_per_user=1, 
                                             num_similar_users=4, 
                                             save_path='cf_predictions.csv', 
                                             rerun_save_path='cf_rerun_predictions.csv', 
                                             seed=RANDOM_STATE):
    """
    Identify failed predictions in CF data and rerun them.

    Args:
    - data: DataFrame containing the original CF predictions.
    - pcc_matrix: Pearson Correlation Coefficient matrix.
    - Other arguments for controlling various aspects of the prediction function.

    Returns:
    - Updated DataFrame with rerun predictions.
    """
    # Ensure the original data has the necessary columns
    if rating_column_name not in data.columns:
        raise KeyError(f"Column '{rating_column}' not found in the data.")

    # Identify rows with failed predictions
    failed_rows = data[pd.to_numeric(data['predicted_rating'], errors='coerce').isna()]

    if len(failed_rows) > 0:
        print(f"Re-running predictions for {len(failed_rows)} failed cases.")

        # Call prediction function on failed data
        rerun_data = predict_ratings_with_collaborative_filtering_and_save(
            failed_rows, pcc_matrix,
            user_column_name=user_column_name,
            movie_column_name=movie_column_name,
            movie_id_column=movie_id_column,
            rating_column_name=rating_column_name,
            num_ratings_per_user=num_ratings_per_user,
            num_similar_users=num_similar_users,
            save_path=rerun_save_path,
            seed=seed
        )

        # Update original data with new predictions
        data.loc[failed_rows.index, 'predicted_rating'] = rerun_data['predicted_rating']

    # Save the updated data
    data.to_csv(save_path, index=False)
    print(f"Updated predictions saved to {save_path}")

    return data



In [ ]:
%%time 

# Load the original CF predictions
cf_data = pd.read_csv(CF_output_path)


# Identify and rerun failed predictions
updated_cf_data = identify_and_rerun_failed_cf_predictions(
    cf_data, pcc_matrix,
    save_path=CF_output_path,
    rating_column_name='actual_rating', 
    rerun_save_path=CF_RERUN_PATH
)



In [ ]:
# Evaluate updated CF model predictions
evaluate_model_predictions_rmse_mae(
    data_path=CF_output_path,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)